In [1]:
import os

In [2]:
%pwd

'c:\\Users\\pachp\\Desktop\\project_git\\chestCancerClassificationDVC\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\pachp\\Desktop\\project_git\\chestCancerClassificationDVC'

In [4]:
# entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:

from chestCancerClassifierDVC.constants import *
from chestCancerClassifierDVC.utils.common_utils import read_yaml, create_directories

In [6]:
# configuration
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,  # redirect the path from constant.yaml
        params_filepath = PARAMS_FILE_PATH):

        # read yaml through path and store it
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # create artifacts root
        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        # config value store here
        config = self.config.data_ingestion

        # create the root dir
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            # initialize configuration
            root_dir=config.root_dir,# where to store
            source_URL=config.source_URL, # url of google drive 
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
# components

import os
import zipfile
import gdown
from chestCancerClassifierDVC import logger
from chestCancerClassifierDVC.utils.common_utils import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
            
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
# pipeline step-wise calling method

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-02 11:22:08,462: INFO: common_utils: yaml file: config\config.yaml loaded successfully]
[2024-03-02 11:22:08,462: INFO: common_utils: yaml file: params.yaml loaded successfully]
[2024-03-02 11:22:08,470: INFO: common_utils: created directory at: artifacts]
[2024-03-02 11:22:08,470: INFO: common_utils: created directory at: artifacts/data_ingestion]
[2024-03-02 11:22:08,479: INFO: 934375234: Downloading data from https://drive.google.com/file/d/1vCPBQkFpH68BnhfdSj3CZw8gsz5pj8CT/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vCPBQkFpH68BnhfdSj3CZw8gsz5pj8CT
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vCPBQkFpH68BnhfdSj3CZw8gsz5pj8CT&confirm=t&uuid=823be2dd-3abe-4161-ae98-b9a8ac2938b3
To: c:\Users\pachp\Desktop\project_git\chestCancerClassificationDVC\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:08<00:00, 5.83MB/s]

[2024-03-02 11:22:18,966: INFO: 934375234: Downloaded data from https://drive.google.com/file/d/1vCPBQkFpH68BnhfdSj3CZw8gsz5pj8CT/view?usp=sharing into file artifacts/data_ingestion/data.zip]
